In [2]:
!pip install transformers evaluate datasets accelerate -q

In [3]:
!pip install --upgrade fsspec datasets

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


Tras la instalación de estas librerías conviene reiniciar el entorno de ejecución. Eso "mata" la sesión de Python pero no la máquina virtual de Colab

### 1 - Datasets

[Repositorio de Datasets](https://huggingface.co/datasets)

Nueva función para listar los datasets (Crea un generador)

In [8]:
from huggingface_hub import list_datasets, dataset_info

all_datasets = list_datasets(sort="downloads", direction=-1, limit=5)

In [6]:
next(all_datasets)

DatasetInfo(id='huggingface/documentation-images', author='huggingface', sha='edccfa0578ed86424c6cfac7ed49f476cc13d441', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2025, 7, 14, 8, 32, 11, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=2830336, downloads_all_time=None, likes=75, paperswithcode_id=None, tags=['license:cc-by-nc-sa-4.0', 'size_categories:n<1K', 'format:imagefolder', 'modality:image', 'library:datasets', 'library:mlcroissant', 'region:us'], trending_score=None, card_data=None, siblings=None, xet_enabled=None)

In [11]:
import pprint

#pprint.pprint(dataset_info('huggingface/documentation-images'))
pprint.pprint(dataset_info('acronym_identification'))

DatasetInfo(id='amirveyseh/acronym_identification',
            author='amirveyseh',
            sha='15ef643450d589d5883e289ffadeb03563e80a9e',
            created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc),
            last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc),
            private=False,
            gated=False,
            disabled=False,
            downloads=880,
            downloads_all_time=None,
            likes=22,
            paperswithcode_id='acronym-identification',
            tags=['task_categories:token-classification',
                  'annotations_creators:expert-generated',
                  'language_creators:found',
                  'multilinguality:monolingual',
                  'source_datasets:original',
                  'language:en',
                  'license:mit',
                  'size_categories:10K<n<100K',
                  'format:parquet',
                  'modality:tex

### 2 - Cargar los datos

In [7]:
from datasets import load_dataset, DatasetInfo

In [9]:
dataset = load_dataset("yelp_review_full")
dataset

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [10]:
dataset["train"][42]

{'label': 4,
 'text': 'What a find! I stopped in here for breakfast while in town for business. The service is so friendly I thought I was down south. The service was quick, frankly and felt like I was with family. \\nFantastic poached eggs, Cajun homefries and crispy bacon. Gab and Eat is definitely a place I world recommend to locals. I was stuffed and the bill was only $8.00.'}

Vamos a usar unos pocos datos, 80 de train y 40 de test. Si no se nos haría eterno el finetunning

In [11]:
# para un mejor entrenamiento utilizar más datos
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(80))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(40))

### 3 - Tokenizer

In [12]:
from transformers import AutoTokenizer

In [14]:
modelo = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(modelo)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

### 4 - Modelo

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 5 - Entrenamiento

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
import numpy as np
import evaluate

In [18]:
metric = evaluate.load("accuracy")

In [19]:
# calculamos el accuracy de las predicciones
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import TrainingArguments, Trainer

In [22]:
# definimos los argumentos del entrenamiento
training_args = TrainingArguments(
    'prueba-modelo',
    eval_strategy="steps",
    logging_steps=5,
    num_train_epochs=1,
    push_to_hub=True,
)

[Argumentos Training](https://huggingface.co/transformers/v4.2.2/main_classes/trainer.html)

In [23]:
# instanciamos el objeto Trainer con todo lo que hemos preparado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Entrenamos...

In [25]:
# 8 min con 80 ejemplos (aprox)
trainer.train()

Step,Training Loss,Validation Loss


Error: You must call wandb.init() before wandb.log()

#### Guardar el modelo en local

In [ ]:
#  trainer.save_model("./modelo")

#### Subir a HuggingFace

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/413M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.81k [00:00<?, ?B/s]

Upload file runs/Jul14_10-25-25_9728fd62ed0b/events.out.tfevents.1689330345.9728fd62ed0b.708.0:   0%|         …

To https://huggingface.co/NechuBM/mi-super-modelo
   ffb3534..1d4745a  main -> main

   ffb3534..1d4745a  main -> main

To https://huggingface.co/NechuBM/mi-super-modelo
   1d4745a..2499fa9  main -> main

   1d4745a..2499fa9  main -> main



'https://huggingface.co/NechuBM/mi-super-modelo/commit/1d4745a50ff39ea486fa10e6f02f466a52dac6f1'

In [ ]:
tokenizer.push_to_hub("NechuBM/mi-super-modelo")

CommitInfo(commit_url='https://huggingface.co/NechuBM/mi-super-modelo/commit/34747d1f6131b88a567b5c67b68b37a733467ec7', commit_message='Upload tokenizer', commit_description='', oid='34747d1f6131b88a567b5c67b68b37a733467ec7', pr_url=None, pr_revision=None, pr_num=None)